<a href="https://colab.research.google.com/github/mosahle7/Generative-AI-with-LLMs/blob/main/W1L1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# %pip install --upgrade pip
# %pip install --disable-pip-version-check \
#     torch==2.0.1 \
#     torchdata==0.6.1 --quiet

# %pip install \
#     transformers==4.41.0 \
#     datasets==2.11.0  --quiet

# %pip install --upgrade torch torchdata transformers datasets


In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [ ]:
# %pip uninstall -y torch torchdata transformers datasets fsspec

# %pip install torch torchdata transformers datasets


  Using cached torch-2.7.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached torchdata-0.11.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached transformers-4.53.3-py3-none-any.whl.metadata (40 kB)
  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.3.0

**Summarize Dialogue Without Prompt Engineering**

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
dataset.shape

In [ ]:
dataset['train'][0]

In [ ]:
dataset['test'][0]

In [ ]:
eg_inds = [40,200]
dash_line = '-'.join('' for x in range(100))

for i,ind in enumerate(eg_inds):
  print(dash_line)
  print('Example ', i+1)
  print(dash_line)
  print('INPUT DIALOGUE: ')
  print(dataset['test'][ind]['dialogue'])
  print(dash_line)
  print('BASELINE HUMAN SUMMARY: ')
  print(dataset['test'][ind]['summary'])
  print(dash_line)
  print('TOPIC:')
  print(dataset['test'][ind]['topic'])
  print()

In [ ]:
# # Cleanly uninstall problematic packages
# %pip uninstall -y transformers torch torchvision torchaudio

# # Reinstall ONLY what's needed for flan-t5 (no torchvision or torchaudio)
# %pip install torch transformers --upgrade


In [ ]:
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

ValueError: Could not find T5ForConditionalGeneration neither in <module 'transformers.models.t5' from '/usr/local/lib/python3.11/dist-packages/transformers/models/t5/__init__.py'> nor in <module 'transformers' from '/usr/local/lib/python3.11/dist-packages/transformers/__init__.py'>!

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
sentence = "What time is it, Tom?"
tokens = tokenizer.tokenize(sentence)
sent_encoded = tokenizer(sentence, return_tensors='pt')
sent_decoded = tokenizer.decode(
    sent_encoded["input_ids"][0],
    skip_special_tokens=True
)

In [ ]:
sent_encoded

In [ ]:
sent_decoded

In [ ]:
tokens

In [ ]:
for i,ind in enumerate(eg_inds):
  dialogue = dataset['test'][ind]['dialogue']
  summary = dataset['test'][ind]['summary']

  inputs = tokenizer(dialogue, return_tensors='pt')
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )

  print(dash_line)
  print('Example ',i+1)
  print(dash_line)
  print(f'INPUT PROMPT:\n {dialogue}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash_line)
  print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')


**Summarize Dialogue with an Instruction Prompt**

**Zero Shot Inference with an Instruction Prompt**

In [ ]:
for i,ind in enumerate(eg_inds):
  dialogue = dataset['test'][ind]['dialogue']
  summary = dataset['test'][ind]['summary']

  prompt = f"""
Summarize the following converstaion:
{dialogue}

Summary:
  """

  inputs = tokenizer(prompt, return_tensors='pt')
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )

  print(dash_line)
  print('Example ', i + 1)
  print(dash_line)
  print(f'INPUT PROMPT:\n{prompt}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash_line)
  print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')



**Zero Shot Inference with the Prompt Template from FLAN-T5**

In [ ]:
for i, index in enumerate(eg_inds):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Dialogue:

{dialogue}

What was going on?
"""
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(

        model.generate(
          inputs["input_ids"],
          max_new_tokens=50,
    )[0],
    skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')


**Summarize Dialogue with One Shot and Few Shot Inference**

In [ ]:
def make_prompt(eg_inds_full, eg_ind_to_sum):
  prompt = ''

  for ind in eg_inds_full:
    dialogue = dataset['test'][ind]['dialogue']
    summary = dataset['test'][ind]['summary']

    prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""

  dialogue = dataset['test'][eg_ind_to_sum]['dialogue']

  prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""

  return prompt

In [ ]:
eg_inds_full = [40]
eg_ind_to_sum = 200

one_shot_prompt = make_prompt(eg_inds_full, eg_ind_to_sum)
print(one_shot_prompt)

In [ ]:
summary = dataset['test'][eg_ind_to_sum]['summary']
input = tokenizer(one_shot_prompt, return_tensors = 'pt')
output = tokenizer.decode(
    model.generate(
      inputs["input_ids"],
      max_new_tokens = 50,
  )[0],
  skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

In [ ]:
eg_inds_full = [40,80,120]
eg_ind_to_sum = 200
few_shot_prompt = make_prompt(eg_inds_full,eg_ind_to_sum)
print(few_shot_prompt)

In [ ]:
input = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50
    )[0],
    skip_special_tokens = True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

**Generative Configuration Parameters for Inference**

In [ ]:
# generation_config = GenerationConfig(max_new_tokens = 50)
# generation_config = GenerationConfig(max_new_tokens = 10)
# generation_config = GenerationConfig(max_new_tokens = 50, do_sample=True, temperature=0.1)
generation_config = GenerationConfig(max_new_tokens = 50, do_sample=True, temperature=0.1)


inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

NameError: name 'tokenizer' is not defined

In [ ]:
l

NameError: name 'l' is not defined